In [ ]:
!pip install langchain_upstage

  Using cached pypdf-4.3.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached orjson-3.10.7-cp310-cp310-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata (50 kB)
Using cached pypdf-4.3.1-py3-none-any.whl (295 kB)
Using cached tiktoken-0.7.0-cp310-cp310-macosx_11_0_arm64.whl (906 kB)
Using cached orjson-3.10.7-cp310-cp310-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl (251 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

In [ ]:
from langchain_upstage import ChatUpstage
from langchain.tools import tool
import json
import os

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

chat = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))

messages = [
  SystemMessage(content="너는 응급구조 요원으로 전화를 받고 있어. 신고자와 소통하여 최대한 빨리 환자의 응급여부를 판단해줘"),
  HumanMessage(content="네 응급구조입니다.")
]
response = chat.invoke(messages)
print(response)

content='안녕하세요, 저는 응급구조 요원입니다. 어떤 도움이 필요하신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 44, 'total_tokens': 62}, 'model_name': 'solar-1-mini-chat-240612', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-25e97698-5924-474b-bebd-2c40f2dfb496-0' usage_metadata={'input_tokens': 44, 'output_tokens': 18, 'total_tokens': 62}


In [ ]:
class EmergencyResponseModel(ollama.BaseModel):
    def __init__(self):
        super().__init__()
        self.dialogue_history = []
        self.tokenizer = ollama.Tokenizer.from_pretrained('skt/kogpt2-base-v2')
        self.model = ollama.Model.from_pretrained('skt/kogpt2-base-v2')

    def add_turn(self, user_input, model_output):
        self.dialogue_history.append({"user": user_input, "model": model_output})

    def get_history(self):
        return self.dialogue_history

In [ ]:
def initial_evaluation(user_input):
    initial_questions = [
        "어디가 아프신가요?",
        "언제부터 아프셨나요?",
        "지금 숨쉬기 괜찮으신가요?",
        "의식이 있나요?",
        "출혈이 있습니까?"
    ]
    if any(symptom in user_input for symptom in ["호흡곤란", "의식 없음", "출혈", "심한 통증"]):
        return "응급", initial_questions
    else:
        return "비응급", ["응급 처치 방법을 알려드리겠습니다."]
    
def generate_prompt(context, dialogue_history):
    prompt = f"Context: {context}\n"
    for turn in dialogue_history:
        prompt += f"User: {turn['user']}\nModel: {turn['model']}\n"
    prompt += "User: "
    return prompt

def generate_response(user_input, dialogue_history):
    if len(dialogue_history.get_history()) == 0:
        urgency, questions = initial_evaluation(user_input)
        if urgency == "응급":
            response = "응급 상황으로 판단됩니다. 추가 질문을 하겠습니다: " + " ".join(questions)
        else:
            response = "응급 상황은 아니지만, 응급 처치 방법을 안내해드리겠습니다."
    else:
        context = "환자는 심각한 상태입니다." if "응급" in dialogue_history.get_history()[0]['model'] else "환자는 비응급 상태입니다."
        prompt = generate_prompt(context, dialogue_history.get_history())
        prompt += user_input + "\nModel: "
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(**inputs, max_length=512, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Model:")[-1].strip()
    
    dialogue_history.add_turn(user_input, response)
    return response


In [2]:
!pip install requests beautifulsoup4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)


In [14]:
import requests
from bs4 import BeautifulSoup

# URL of the blog page
url = 'https://doraemong0808.tistory.com/entry/KTAS%EB%9E%80?category=1141799'

# Send a GET request to fetch the HTML content
response = requests.get(url)
if response.status_code == 200:
    html_content = response.content

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract post content
    content = soup.find('div', class_='tt_article_useless_p_margin contents_style').text.strip()

    # Print the content (optional)
    print(f"Content: {content}")

    # Save the content to a text file
    with open('ktas_info.txt', 'w', encoding='utf-8') as file:
        file.write(content.replace('☞',' '))


    print("Content has been saved to 'ktas_info.txt'")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


Content: KTAS란?(korean triage and acuity scale) 1. 정의 KTAS(Korean Triage and Acuity Scale)는 한국형 응급환자 분류도구를 의미하는 영어 줄임말입니다. 2012년 캐나다 응급환자 분류도구인 CTAS(Canadian Triageand Acuity Scale)를 우리나라 의료상황에 맞게 변형하여 KTAS를 개발하였습니다. KTAS는 단순히 응급실에 내원하는 환자의 진료 순서를 정하기 위한 목적 이외에도 병원 전 단계를 아울러서 응급의료체계 전반을 개혁하기 위해 시작되었습니다. 응급환자 중증도 분류(Triage)는 응급실 간호사의 업무 중 가장 전문적인 역할로 의학지식과 판단력을 요구한다. 응급실에서 환자를 분류하고 우선순위에 따라 의료조치를 취하기 위해 사용되는 도구입니다. 원활한 응급의료체계 확립을 위해 보건복지부는 표준화된 한국형 중증도 분류도구(KTAS: Korean Triage and Acuity Scale)를 개발하여 2016년 1월1일부터 전국 응급의료센터에서 시행하고 있습니다. 2. KTAS 자격 KTAS 자격을 취득한 응급의료기관 종사자 중 직종별 통계 상 높은 비율로 간호사가 KTAS를 수행하고 있습니다. KTAS는 [응급의료에 관한 법률 시행규칙 제18조의 제3조 2항, 한국 응급환자 중증도분류기준]에 의거하여 시행주체를 의사, 간호사, 1급 응급구조사의 면허 또는 자격을 갖춘 자로 정의하고 이중 최근 5년 이내의 1년 이상(만 12개월)의 응급실 경험이 있고, KTAS 교육과정을 이수 및 자격을 취득한 자가 시행할 수 있다(보건복지부, 2015; KTAS위원회, 2021). 특히 KTAS 자격을 이수한 응급의료기관 종사자 중 직종별 통계상 간호사의 비율이 91.9%로 평가되었으며 많은 기관에서 응급실 간호사가 직종별 통계 상 높은 비율로 중증도 분류 업무를 담당하고 있는 것이 확인되었으며, 그 외 응급구조사가 5.5%, 의사가 2.6% 로평가한다고 보고되고 있습니다. 3. KT

In [13]:
content.replace('☞','').split(u'\xa0')

['KTAS란?(korean triage and acuity scale)',
 '1. 정의',
 'KTAS(Korean Triage and Acuity Scale)는 한국형 응급환자 분류도구를 의미하는 영어 줄임말입니다. 2012년 캐나다 응급환자 분류도구인 CTAS(Canadian Triageand Acuity Scale)를 우리나라 의료상황에 맞게 변형하여 KTAS를 개발하였습니다. KTAS는 단순히 응급실에 내원하는 환자의 진료 순서를 정하기 위한 목적 이외에도 병원 전 단계를 아울러서 응급의료체계 전반을 개혁하기 위해 시작되었습니다. 응급환자 중증도 분류(Triage)는 응급실 간호사의 업무 중 가장 전문적인 역할로 의학지식과 판단력을 요구한다. 응급실에서 환자를 분류하고 우선순위에 따라 의료조치를 취하기 위해 사용되는 도구입니다. 원활한 응급의료체계 확립을 위해 보건복지부는 표준화된 한국형 중증도 분류도구(KTAS: Korean Triage and Acuity Scale)를 개발하여 2016년 1월1일부터 전국 응급의료센터에서 시행하고 있습니다.',
 '2. KTAS 자격',
 'KTAS 자격을 취득한 응급의료기관 종사자 중 직종별 통계 상 높은 비율로 간호사가 KTAS를 수행하고 있습니다. KTAS는 [응급의료에 관한 법률 시행규칙 제18조의 제3조 2항, 한국 응급환자 중증도분류기준]에 의거하여 시행주체를 의사, 간호사, 1급 응급구조사의 면허 또는 자격을 갖춘 자로 정의하고 이중 최근 5년 이내의 1년 이상(만 12개월)의 응급실 경험이 있고, KTAS 교육과정을 이수 및 자격을 취득한 자가 시행할 수 있다(보건복지부, 2015; KTAS위원회, 2021). 특히 KTAS 자격을 이수한 응급의료기관 종사자 중 직종별 통계상 간호사의 비율이 91.9%로 평가되었으며 많은 기관에서 응급실 간호사가 직종별 통계 상 높은 비율로 중증도 분류 업무를 담당하고 있는 것이 확인되었으며, 그 외 응급구조사가 5.5%, 의사가 2.6% 로평가한다고 보고되고 있습